<p style="font-weight:bold; font-size: 40px; color:#0171ff; padding-top: 50px; text-align:center"> How to compute Present Values </p>

> IFRS 17 is a new accounting standard for insurance contracts. It is an economic accounting approach, replacing the nominal accounting of the previous standard IFRS 4, that is, the economic value of the insurance products is considered. The focus is on the liabilities of an insurance company, namely the insurance policies. 

</p>

<br/>

The aim of this notebook is to illustrate the IFRS 17 *Present Value* calculation using Systemorph Cloud Technology. Present Values are the amount of money that someone would pay in the present day for the contracts of the group up to their run off. The starting point are the so called *Nominal Cash flows*, which express the amounts of cash and cash equivalents being transferred into and out of a business. Cash flow values are **discounted** according to the *Yield Curve* provided as economic input in order to take into account the corresponding Interest Accretion, and the discounted figures are **cumulated** to find the **Present Values**. 

The IFRS 17 standard prescribes that the accounting statements are based on the value of a group of insurance contracts at the beginning of the period (BoP), their development throughout the period, and the value at the end of the period (EoP). While the period is typically a quarter, the **Analysis of Change** (AoC) from BoP to EoP per group of contract is made considering for each step the variation of the cash flow so as to enhance the readability of the value changes. To this aim it is necessary to **evaluate the Present Value difference** (or delta) between each step, yielding the figures shown in the resulting report.

This process is pictorially represented in the flowchart below

<center><img src="https://stacdnsmcwe.blob.core.windows.net/content/IFRS17CalculationEngine/Images/PresentValueOfCashFlow/Flowchart.png" width="40%" style="float:center;">

---

Import the IFRS 17 calculation engine:

In [0]:
#!import "//ifrs17/v1.0.0/CalculationEngine"

Initialize basic pre-configured dimensions that can be tuned to reflect for your company and business. Example of such dimensions are

**Reporting Nodes**: hierarchical structure of the company, where the root is the *Group* level and the end nodes are the levels at which data is imported.

**Data Nodes**: Grouping of (Re-)Insurance Contracts belonging to the same Portfolio, and defined by their Reporting Node, Scenario, Contractual Currency, FunctionalCurrency, LineOfBusiness, ValuationApproach, and OCI Type. 

**AoC Type**: the Analysis of Change steps used for accounting statements.

In [0]:
#!eval-notebook "InitializeData"

---

# Import

In the following code cells, the necessary data to start the IFRS calculator are being imported.
After the standard dimensions and parameters are loaded, for the calculation of the Present Value 
one needs the economic input yield curves for the target period
and the nominal cash flows for the desired group of contracts:

**Yield Curve**: line that plots yields, i.e. interest rates. It depends on the given currency related to the target *Reporting Node*, and is imported on a yierly basis. The slope of the yield curve gives an idea of future interest rate changes and economic activity.

**Nominal Cash flows**: they are the amounts of cash and cash equivalents that a company expects to transfer into and out of a business, without any adjustment. This is useful for anticipating future revenue and expenses.

The import process of these example spreadsheets can be completed by running the two cells below

In [0]:
await Import.FromFile("DataNodes_CH.xlsx")
            .WithFormat("DataNode")
            .WithTarget(DataSource)
            .ExecuteWithStoreActivityAsync(Session, DataSource, DataSetReader)

In [0]:
await Import.FromFile("DataNodes_DE.xlsx")
            .WithFormat("DataNode")
            .WithTarget(DataSource)
            .ExecuteWithStoreActivityAsync(Session, DataSource, DataSetReader)

In [0]:
await Import.FromFile("YieldCurve.xlsx")
            .WithType<YieldCurve>()
            .WithTarget(DataSource)
            .ExecuteWithStoreActivityAsync(Session, DataSource, DataSetReader)

In [0]:
await Import.FromFile("Cashflows.xlsx")
            .WithFormat("Cashflow")
            .WithTarget(DataSource)
            .ExecuteWithStoreActivityAsync(Session, DataSource, DataSetReader)

In [0]:
await Import.FromFile("CF_CH_2021_12.xlsx")
            .WithFormat("Cashflow")
            .WithTarget(DataSource)
            .ExecuteWithStoreActivityAsync(Session, DataSource, DataSetReader)

In [0]:
await Import.FromFile("CF_DE_2021_12.xlsx")
            .WithFormat("Cashflow")
            .WithTarget(DataSource)
            .ExecuteWithStoreActivityAsync(Session, DataSource, DataSetReader)

In [0]:
await Import.FromFile("CF_DE_2022_12.xlsx")
            .WithFormat("Cashflow")
            .WithTarget(DataSource)
            .ExecuteWithStoreActivityAsync(Session, DataSource, DataSetReader)

In [0]:
ifrs17.Reset(Workspace)

<br/><br/><br/><br/>

<a id='present-value-report'></a>
# Present Value Report

Present Value is today’s value of money you expect from future income and is calculated as the sum of future investment returns discounted at a specified level of rate of return expectation.

The change of the Present Value in the target period is analysed following the Analysis of Change method.

In [0]:
var pv = ifrs17.PresentValues;

In [0]:
pv.ReportingNode = "DE";
pv.ReportingPeriod = (2022, 12);
pv.ColumnSlices = new[] {"ReportingNode", "EstimateType", "AmountType"};
pv.DataFilter = new[] {("EconomicBasis", "L")};

In [0]:
await pv.ToReportAsync

<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>

# View imported Data

Systemorph notebook technology excels in analyse on the fly the imported data. In this section the newly imported Yield Curves and nominal cash flows can be analysed.

**Imported Yield Curve**

In [0]:
var yieldCurves = await DataSource.Query<YieldCurve>().ToArrayAsync();

In [0]:
await Report.ForDataCube(yieldCurves.ToReportType()).SliceRowsBy("Currency").SliceColumnsBy("Index").ToLineChart().ExecuteAsync() with { Width = 1000 }

**Imported Nominal Cash flows**

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>( new {ReportingNode = pv.ReportingNode, Year = pv.ReportingPeriod.Year, Month = pv.ReportingPeriod.Month, Scenario = (string)null} );

In [0]:
var nominals = (await DataSource.Query<RawVariable>().ToArrayAsync()).ToDataCube();

In [0]:
await Report.ForDataCube(nominals.ToReportType()).SliceRowsBy("AmountType","AocType").SliceColumnsBy("Index").WithQuerySource(Workspace).ToBarChart().ExecuteAsync()

# Discounting and Cumulate

The list of **Amount Types** are retrieved from the Data Source

In [0]:
var amountTypes = await DataSource.Query<AmountType>().ToArrayAsync();

The **Yield Curve** associated to the target Group of Contract Reporting Node Currency is selected for the computation of the Discount rates

$$
\text{Discount}_i = \big( 1 + \text{YieldCurve}_i \big) ^{-\frac{1}{12}} ~.
$$

In [0]:
var discountRates = ( from yc in yieldCurves.FirstOrDefault(x => x.Currency == "CHF").Values select Math.Pow((1 + yc),( - 1.0 / 12.0 )) ).ToArray();

Before discounting the cash flows, it is necessary to match the discount rates starting period with the target period selected in the [report](#present-value-report). 
Since the imported Yearly Yield Curve starts from the year 2021, we can simply skip the past elements with the aid of the following cell 

In [0]:
discountRates = discountRates.Skip(pv.ReportingPeriod.Year - 2021).ToArray();

The Cumulated Discounted Cash flow ($\text{CDC}$) is defined by the following recursive formulas

$$
\text{CDC}_t = \left\{
\begin{array}{cl}
\text{Nominal}_t + \text{CDC}_{t+1} \cdot {\text{Valid Discount}_{\frac{t}{12}}} ~, & \text{if Period Type is Beginning of Period} \\
\big( \text{Nominal}_t + \text{CDC}_{t+1} \big) \cdot {\text{Valid Discount}_{\frac{t}{12}}} ~, & \text{if Period Type is End of Period}
\end{array}
\right.
$$

where the Period Type depends on the given cash flow Amount Type. 

In the following, the premiums are retrieved from the nominals **Data Cube** through a simple Filter. 
Additionally, among the list of premiums provided, we select only the cash flows for the Assumption Update (AU) and Experience Variance (EV) steps.

The discount and cumulation operation can be performed on the resulting Data Cube through the method <code>ComputeDiscountAndCumulateAsync</code> provided by the solution. We focus on the Assumption Update step

In [0]:
var premiumsAU = nominals.Filter(("AmountType", "PR"),("AocType", "AU"));

In [0]:
var premiumsAU_CDC = premiumsAU.ComputeDiscountAndCumulate( discountRates, amountTypes );

and on the Experience Variance step

In [0]:
var premiumsEV = nominals.Filter(("AmountType", "PR"),("AocType", "EV"));

In [0]:
var premiumsEV_CDC = premiumsEV.ComputeDiscountAndCumulate( discountRates, amountTypes );

# Evaluating the Delta for a given step

In [0]:
var premiums_Delta_EV_AU = (premiumsEV_CDC - premiumsAU_CDC).Aggregate();

The contribution to the Present Value related to the Experience Variance AoC Step, is given by the n-th element of the Values array, 
with $n$ equal to the target Month. This element corresponds to the value at the EOP of the current quarter, that is, the BOP of the next one

In [0]:
-premiums_Delta_EV_AU.Values[12]

This Delta Present Value can then be compared with the value provided in the [reporting table](#present-value-report). 

Note that the minus sign is needed to reconcile with the reserve view of the reports. 

---